## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе;

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

from mystem import MyStem

C:\Users\PC\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
_mystem = MyStem()

## Данные

In [4]:
stops = set(stopwords.words('russian')) | {'gt',}
def remove_tags(text):
    return re.sub(r'<[^>]+>|gt', '', text)

def normalize(words):
    norm_words = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return norm_words

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [5]:
data_raw = open('habr_texts.txt', encoding='utf-8').read().splitlines()

In [6]:
data_raw = [remove_tags(doc) for doc in data_raw]

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [8]:
%%time
data_norm = _mystem.run(data_raw, flags='-idln', remove_stopwords=True)

Wall time: 2min 4s


In [9]:
data_norm = [doc.split() for doc in data_norm]

### Тематическое моделирование в gensim

In [16]:
def lda_gensim(data, freq_thresh=0.3, ngrams_thresh=0.4, tfidf=False, model_params=None):
    _model_params = {
        'num_topics': 10,
        'workers': 3,
        'passes': 10,
        'eta': 'auto',
        'iterations': 20
    }
    _data = data
    
    if model_params is not None:
        _model_params.update(model_params)
    
    if ngrams_thresh is not None:
        ph = gensim.models.Phrases(_data, scoring='npmi', threshold=ngrams_thresh)
        p = gensim.models.phrases.Phraser(ph)
        _data = p[_data]
    
    dictionary = gensim.corpora.Dictionary(_data)
    dictionary.filter_extremes(no_above=freq_thresh)
    dictionary.compactify()
    
    corpus = [dictionary.doc2bow(text) for text in _data]
    
    if tfidf:
        _tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
        corpus = _tfidf[corpus]
        
    lda = gensim.models.LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            random_state=17,
            **_model_params
    )
    
    return lda
    

Построим модель на 10 тем

In [35]:
%%time
lda10 = lda_gensim(data_norm)

Wall time: 1min 5s


Посмотрим на темы

In [36]:
lda10.print_topics()

[(0,
  '0.026*"lt" + 0.009*"if" + 0.007*"return" + 0.006*"name" + 0.005*"i" + 0.005*"id" + 0.005*"this" + 0.004*"int" + 0.004*"файл" + 0.004*"класс"'),
 (1,
  '0.007*"сайт" + 0.004*"товар" + 0.003*"игрок" + 0.003*"клиент" + 0.003*"скидка" + 0.003*"день" + 0.003*"страница" + 0.003*"игра" + 0.003*"участник" + 0.003*"интерфейс"'),
 (2,
  '0.009*"устройство" + 0.004*"модель" + 0.003*"смартфон" + 0.003*"камера" + 0.003*"вселенная" + 0.003*"технология" + 0.002*"энергия" + 0.002*"производитель" + 0.002*"модуль" + 0.002*"сигнал"'),
 (3,
  '0.004*"объект" + 0.004*"точка" + 0.002*"изображение" + 0.002*"размер" + 0.002*"элемент" + 0.002*"скорость" + 0.002*"земля" + 0.002*"материал" + 0.002*"печать" + 0.002*"модель"'),
 (4,
  '0.006*"and" + 0.005*"игра" + 0.005*"the" + 0.004*"of" + 0.004*"to" + 0.004*"for" + 0.003*"файл" + 0.003*"доклад" + 0.003*"VR" + 0.003*"a"'),
 (5,
  '0.012*"игра" + 0.005*"сервер" + 0.003*"какой-то" + 0.003*"память" + 0.003*"игрок" + 0.003*"производительность" + 0.002*"програ

В целом темы кажутся осмысленными, однако во многих из них повторяется веб-тематика.  
Помимо этого, некоторые темы забиты кодом, которого в данных очень много.  
Возможно, для повышения качества тем следует избавиться от кода в данных и несколько изменить порог частотности при построении словаря 

Так как в данных участки кода никак не помечены, я не вижу никакого иного способа избавиться от кода, кроме как удалить латинские символы

In [11]:
data_norm2 = [
    re.sub('[a-zA-Z]', '', ' '.join(doc)).split() for doc in data_norm
]

Построим теперь модели на 10, 25, 50 и 100 тем со сниженным порогом вероятности и `npmi`

In [17]:
%%time
lda_res = {
    i: lda_gensim(data_norm2, freq_thresh=0.25, ngrams_thresh=0.3, model_params={'num_topics': i, 'passes': 20})
    for i in [10, 25, 50, 100]
}

Wall time: 6min 3s


Посмотрим на них (на топ-10 самых важных для каждой модели)

In [18]:
for i, lda in lda_res.items():
    print('%s:\n%s\n' % ('LDA %s' % i, lda.print_topics(10)))

LDA 10:
[(0, '0.007*"клиент" + 0.007*"сеть" + 0.006*"сервис" + 0.005*"сервер" + 0.004*"услуга" + 0.004*"безопасность" + 0.004*"бизнес" + 0.003*"управление" + 0.003*"трафик" + 0.003*"инфраструктура"'), (1, '0.006*"алгоритм" + 0.005*"объект" + 0.005*"элемент" + 0.005*"модель" + 0.004*"точка" + 0.003*"реализация" + 0.003*"класс" + 0.003*"изображение" + 0.003*"тестирование" + 0.003*"подход"'), (2, '0.007*"сообщение" + 0.006*"модель" + 0.006*"страница" + 0.005*"запрос" + 0.005*"бот" + 0.005*"элемент" + 0.004*"сервис" + 0.004*"документ" + 0.004*"текст" + 0.004*"сеть"'), (3, '0.007*"класс" + 0.006*"запрос" + 0.006*"объект" + 0.005*"библиотека" + 0.005*"строка" + 0.005*"компонент" + 0.004*"модуль" + 0.004*"элемент" + 0.004*"параметр" + 0.004*"память"'), (4, '0.013*"сервер" + 0.007*"настройка" + 0.005*"адрес" + 0.005*"скрипт" + 0.004*"установка" + 0.004*"клиент" + 0.003*"настраивать" + 0.003*"сервис" + 0.003*"указывать" + 0.003*"пароль"'), (5, '0.003*"страна" + 0.003*"мир" + 0.003*"робот" + 0.0

Видно, что качество тем улучшилось и каждая модель способна выделить осмысленные темы касательно веб-разработки, струтктурного программирования, машинного обучения, техники и устройств, бизнеса и даже карьерного развития.  
Также начали появляться устойчивые словосочетания, которых вообще не было ранее.  
Как мне кажется, тремя наиболее интересными и хорошими темами оказались темы LDA25 модели:
- бэкэнд-разработка:
(18, '0.013*"сервер" + 0.013*"т_е" + 0.009*"запрос" + 0.007*"база__данные" + 0.005*"сервис" + 0.005*"очередь" + 0.004*"токен" + 0.004*"либо" + 0.003*"клиент" + 0.003*"сессия"')

- машинное обучение:
(2, '0.015*"модель" + 0.010*"сеть" + 0.009*"обучение" + 0.008*"бот" + 0.007*"изображение" + 0.007*"признак" + 0.006*"нейрон" + 0.005*"алгоритм" + 0.005*"слой" + 0.005*"обучать"')

- полеты в космос:
(12, '0.010*"спутник" + 0.006*"ракета" + 0.005*"запуск" + 0.005*"двигатель" + 0.004*"пуск" + 0.004*"орбита" + 0.003*"космос" + 0.003*"транзистор" + 0.003*"ракета_носитель" + 0.003*"космический"')

Касательно изменений в сравнении с предыдущим пайплайном, по LDA10 модели видно, что темы стали лучше, более полноценными и независимыми друг от друга, появились устойчивые словосочетания, нейтрализовалась доминация веб тематики и темы более не засорены кодом. Также появились и новые полноценные темы, например, космоса или карьерного роста (что интересно получилось (тема 8))  

Добавим теперь `tfidf` и повторим эксперимент 

In [19]:
%%time
lda_res2 = {
    i: lda_gensim(
        data_norm2,
        freq_thresh=0.25,
        ngrams_thresh=0.3,
        tfidf=True,
        model_params={'num_topics': i, 'passes': 20})
    for i in [10, 25, 50, 100]
}

Wall time: 16min 59s


In [20]:
for i, lda in lda_res2.items():
    print('%s:\n%s\n' % ('LDA %s' % i, lda.print_topics(10)))

LDA 10:
[(0, '0.001*"игра" + 0.000*"ä_ä" + 0.000*"ä" + 0.000*"сервис" + 0.000*"объект" + 0.000*"продукт" + 0.000*"клиент" + 0.000*"сеть" + 0.000*"модель" + 0.000*"сервер"'), (1, '0.001*"игра" + 0.000*"сервис" + 0.000*"объект" + 0.000*"продукт" + 0.000*"клиент" + 0.000*"сеть" + 0.000*"модель" + 0.000*"сервер" + 0.000*"блок" + 0.000*"книга"'), (2, '0.001*"игра" + 0.000*"сервис" + 0.000*"объект" + 0.000*"клиент" + 0.000*"продукт" + 0.000*"сеть" + 0.000*"модель" + 0.000*"сервер" + 0.000*"элемент" + 0.000*"класс"'), (3, '0.001*"игра" + 0.000*"сервис" + 0.000*"объект" + 0.000*"клиент" + 0.000*"продукт" + 0.000*"сеть" + 0.000*"модель" + 0.000*"сервер" + 0.000*"класс" + 0.000*"элемент"'), (4, '0.001*"игра" + 0.001*"сервер" + 0.001*"объект" + 0.001*"клиент" + 0.001*"запрос" + 0.001*"класс" + 0.001*"сервис" + 0.001*"библиотека" + 0.001*"модуль" + 0.001*"страница"'), (5, '0.001*"игра" + 0.000*"сервис" + 0.000*"объект" + 0.000*"продукт" + 0.000*"клиент" + 0.000*"сеть" + 0.000*"модель" + 0.000*"сер

Добавление `tfidf`, к сожалению, не позволило улучшить качество, но, наоборот, только ухудшило.  
Появились совершенно бессмысленные темы, а во многих осмысленных появилось немало шума.  
Но некоторые сильные тематики, такие, как веб- и структурная разработка, по-прежнему выделяются довольно хорошо и полноценно.

### Разложение матриц в sklearn

In [21]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [22]:
data_norm3 = [' '.join(text) for text in data_norm2]

In [36]:
def nmf(data, n_components=10, vectorizer_params=None, alpha=0):
    _vectorizer_params = {
        'max_features': 10000,
        'min_df': 10,
        'max_df': 0.2,
        'lowercase': False
    }
    
    if vectorizer_params is not None:
        _vectorizer_params.update(vectorizer_params)
        
    vectorizer = TfidfVectorizer(**_vectorizer_params)
    X = vectorizer.fit_transform(data)
    
    model = NMF(n_components, random_state=17, alpha=alpha)
    model.fit(X)
    
    return model, vectorizer

def get_nmf_topics(model, vectorizer, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(model.n_components):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

Запустим `NMF` на 10, 25, 50 и 100 тем, а также несколько изменим частотные параметры векторизатора, также добавив биграммы. 

In [31]:
%%time
nmf_res = {
    i: nmf(data_norm3, n_components=i, vectorizer_params={'ngram_range': (1, 2)})
    for i in [10, 25, 50, 100]
}

Wall time: 2min 4s


Посмотрим на темы

In [38]:
for i, _nmf in nmf_res.items():
    print('%s:\n%s\n' % ('NMF %s' % i, get_nmf_topics(*_nmf, 10)))

NMF 10:
   Topic # 01    Topic # 02  Topic # 03    Topic # 04          Topic # 05  \
0      бизнес          игра       класс      обучение         виртуальный   
1       рынок         игрок      запрос            ия                диск   
2   сотрудник       игровой      строка          мозг              машина   
3      услуга        играть  библиотека         робот  виртуальный машина   
4    блокчейн      персонаж   компонент        ученый              память   
5      деньги        движок    страница     нейросеть           резервный   
6     продажа  игра который      модуль      алгоритм                  ВМ   
7  российский     мобильный       вызов  исследование              облако   
8   мобильный          жанр     таблица     интеллект  производительность   
9      страна     приставка        тест     нейронный                ядро   

    Topic # 06     Topic # 07        Topic # 08   Topic # 09    Topic # 10  
0       камера     уязвимость             книга  космический       

Результаты `NMF` кажутся невероятно хорошими, можно даже сказать удивительными.  
Каждая из моделей выделяет полноценные, точные и лишенные шума темы, при этом все темы крайне осмысленны и не пересекаются друг с другом.  
Помимо этого, поражает их разнообразие и, самое главное, сохраняющаяся при этом точность, полноценность и гомогенность даже самых узких тем (от кибер-безопасности и VR до тематики конференций или видеорегистраторов).  
Думаю, здесь нет смысла пытаться выделять топ-3 темы, потому что разнообразие, точность и наполнение тем поражает, все они очень хороши.  
Как мне кажется, (на этих данных, по крайней мере, и при прочих равных) `NMF` колоссально превосходит по качеству `LDA`, настолько, что они едва ли кажутся сравнимыми.